<a href="https://colab.research.google.com/github/ge28yen/Complex-Physics-with-GNNs/blob/main/Copy_of_GNN_Physics_my_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Import dependencies, set up configuration



In [ ]:
import torch.nn as nn
import torch
from typing import *
import logging
from typing import *
import numpy as np
import math
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import wandb

# Install torch geometric
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 101.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.0 MB/s eta 0:00:00


In [ ]:
import torch_geometric as pyg
import torch_scatter
from google.colab import drive
drive.mount('/content/drive') # WARNING: the dataset is currenlty only available on the

Mounted at /content/drive


In [ ]:
# Configure the logging level and format
logging.basicConfig(
    level=logging.ERROR,  #set to logging.INFO if you want the debugging messages shown, logging.ERROR otherwise
    format="%(asctime)s - %(levelname)s - %(message)s",
    force=True
)

METADATA = {
    'epochs': 1,
    'learning_rate': 0.005,
    'batch_size': 4, #even this batch size overloads the available GPU sometimes
    'connectivity_radius': 0.02,
    'borders_x' : [0.1, 0.9],
    'borders_y' : [0.1, 0.9],
    'timestep' : 1, #time lag duration
    'embedding_dimension': 16,
    'hidden_dimension': 128,
    "dt": 0.0025,
    "vel_mean": [1.1927917091800243e-05, -0.0002563314637168018],
    "vel_std": [0.0013973410613251076, 0.00131291713199288],
    "acc_mean": [-1.10709094667326e-08, 8.749365512454699e-08],
    "acc_std": [6.545267379756913e-05, 7.965494666766224e-05]
}

In [ ]:
wandb.login(key="")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 1. Define the Dataset

## Data preprocessing
The data in the form of needs to be preprocessed. The input graph to to the model should have the following attributes:
- graph.x : an array of ints indicating particle types
- graph.y : an array of n_dim - dimensional

In [ ]:
def velocities_from_positions(positions_seq, timestep):
    #Calculation: velocity_i = position_i - position_i-1 / timestep

    velocities = positions_seq[:, 1:, :] - positions_seq [:, :-1, :]
    logging.info(f'velocities shape: {velocities.shape}')

    return velocities

def recalculate_positions(target_positions, x_boundaries, y_boundaries):
    #Calculate distances to boundaries from particle positions

    x_target_positions = target_positions[:, 0, 0].squeeze()
    y_target_positions = target_positions[:, 0, 1].squeeze()
    x_boundaries = torch.stack((x_target_positions-x_boundaries[0], x_boundaries[1]-x_target_positions), dim=1) # shape = (400, 2),
    y_boundaries = torch.stack((y_target_positions-y_boundaries[0], y_boundaries[1]-y_target_positions), dim=1)
    logging.info(f'x_boundaries shape: {x_boundaries.shape}')
    logging.info(f'y_boundaries shape: {y_boundaries.shape}')

    return x_boundaries, y_boundaries

def acceleration_from_velocities(velocities, timestep):
    #Calculate ast acceleration from velocities

    acceleration = (velocities[:, 1:] - velocities[:, :-1])/timestep
    logging.info(f'acceleration shape, {acceleration.shape}')
    last_acceleration = acceleration[:,-1] # TODO: this can be done more efficient
    logging.info(f'last acceleration, {last_acceleration[:5]}')

    return last_acceleration
    # Calculate accelerations from velocities

def get_edge_features(edge_indexes, target_positions):
    #Calculate relative displacements and absolute distance of the edges

    transposed_edge_indexes = torch.t(edge_indexes)
    edge_features = []
    for two_nodes in transposed_edge_indexes.numpy():
      node1 = two_nodes[0]
      node2 = two_nodes[1]
      position_node1 = target_positions.squeeze().numpy()[node1]
      position_node2 = target_positions.squeeze().numpy()[node2]
      relative_position = [position_node1[0] - position_node1[0], position_node2[1] - position_node1[0]]
      distance = math.sqrt(sum(x**2 for x in relative_position))
      relative_position.append(distance)
      edge_features.append(relative_position)
    edge_features = torch.tensor(edge_features)
    logging.info(f'edge_features_shape, {edge_features.shape}')

    return edge_features

def preprocess(particle_type, positions_seq ,metadata): # tensors of shape (n_particles), (n_particles, n_timesteps, dim),

  #0. preprocess the postion_sequence:
  target_positions = positions_seq[:, -1: :]
  previous_positions = positions_seq[:, :-1]

  #1.Calculate velocities from previous_positons
  timestep = metadata['timestep']
  velocities = velocities_from_positions(positions_seq, timestep)

  #2.Recalculate positons as given boundaries
  borders_x = metadata['borders_x']
  borders_y = metadata['borders_y']
  x_boundaries, y_boundaries  = recalculate_positions(target_positions, borders_x, borders_y)

  #3. Calculate the edge indexes:
  target_positions.squeeze()
  edge_indexes = pyg.nn.radius_graph(target_positions.squeeze(), metadata['connectivity_radius'])
  logging.info('edge_indexes shape, {edge_indexes.shape}')       # Should have shape (2, n_edges)


  #4. Calculate the edge features:
  edge_features = get_edge_features(edge_indexes, target_positions)

  #5. Calculate the accelerations:
  acceleration = acceleration_from_velocities(velocities, timestep)

  ## 5. Sum it all up in a graph:
  flattened_velocities = velocities.view(velocities.shape[0], -1)

  graph = pyg.data.Data(
      x = particle_type,
      edge_index = edge_indexes,
      node_features =torch.cat((x_boundaries, y_boundaries, flattened_velocities), dim =-1 ),#torch.cat(None, dim = -1)
      edge_features =edge_features,
      y = acceleration
  )

  return graph

## Define the Datasets

In [ ]:
#check the file loading
base_path = '/content/drive/MyDrive/GGN_for_physics_DATA/'
valid_pth_path = base_path +"/valid_dataset.pth"
valid_json_path = base_path +"/valid_offsets.json"
test_pth_path = base_path + "/test_dataset.pth"
test_json_path = base_path + "/test_offsets.json"


In [ ]:
import json

class ShortDataset(torch.utils.data.Dataset):
    def __init__(self, pth_path, offsets_path):
        super().__init__()
        self.dataset = torch.load(pth_path)
        with open(offsets_path, 'rb') as f:
          self.offsets = json.load(f)

        logging.info(f'self.offsets, {self.offsets}')
        self.length = int(list(self.offsets.keys())[-1]) #the length will be the number of the last

    def __len__(self):
        return 763

    def __getitem__(self, idx):
        if idx>2990:
          return None
        offset_id = self.offsets[str(idx)]
        particle_type_offset = offset_id['particle_type']['offset']
        position_offset = offset_id['position']['offset']

        shape =offset_id['position']['shape']
        n_particles = shape[0]
        positions = self.dataset['position'][position_offset:position_offset+n_particles]
        particle_types = self.dataset['particle_type'][particle_type_offset:particle_type_offset +n_particles]
        graph  = preprocess(particle_types, positions, METADATA)
        return graph


In [ ]:
#test the Short Dataset
valid_short_dataset = ShortDataset(valid_pth_path, valid_json_path)
test_short_dataset = ShortDataset(test_pth_path, test_json_path)

print(len(valid_short_dataset))
graph = test_short_dataset[1278]


<ipython-input-63-290bc56fe217>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.dataset = torch.load(pth_path)


763


In [ ]:
positions.shape

NameError: name 'positions' is not defined

## Visualize the graph

In [ ]:
# Visualize a datapoint:
import matplotlib.pyplot as pyplot
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib import animation
from IPython.display import HTML
%matplotlib notebook
# Ensure ffmpeg is installed
!apt-get install -y ffmpeg

# Assuming `positions` is already defined and contains the correct shape
if True:
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML

    # Prepare your data
    new_positions = positions.transpose(0, 1)  # Your actual data
    data = new_positions.numpy()  # Convert to numpy array if it's a tensor

    # Set up the figure and axis
    fig, ax = plt.subplots()
    ax.set_xlim(np.min(data[:, :, 0]), np.max(data[:, :, 0]))
    ax.set_ylim(np.min(data[:, :, 1]), np.max(data[:, :, 1]))
    sc = ax.scatter([], [], s=10)

    # Initialization function
    def init():
        sc.set_offsets(np.empty((0, 2)))  # Empty 2D array for initialization
        return sc,

    # Update function
    def update(frame):
        offsets = data[frame]  # Extract frame data of shape (n, 2)
        sc.set_offsets(offsets)
        return sc,

    # Create the animation
    ani = FuncAnimation(
        fig, update, frames=data.shape[0], init_func=init, blit=True, interval=100
    )

    # Render and display the animation in Colab
    display(HTML(ani.to_html5_video()))

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


NameError: name 'positions' is not defined

In [ ]:
import json
import torch
from torch.utils.data import Dataset

class RolloutDataset(Dataset):
    """
    Interprets each entry in `offsets.json` as one rollout of shape
    [n_particles, 1000, 2]. The final returned shape is by default
    [1000, n_particles, 2] if we transpose time to the first dimension.

    For each index i, returns:
        {
            "particle_type": <long tensor, shape: (n_particles,)>,
            "position":      <float32 tensor, shape: (1000, n_particles, 2)>,
        }
    """

    def __init__(self, pth_path, offsets_path, transpose_to_time_first=True):
        super().__init__()

        # NOTE: If you trust the pth file, leave weights_only=False as default,
        # otherwise you can do: torch.load(pth_path, weights_only=True)
        # to avoid the PyTorch warning about pickle.
        self.dataset = torch.load(pth_path)

        # Load offsets from JSON
        with open(offsets_path, "r") as f:
            self.offsets = json.load(f)

        # The dataset length is the total number of rollout entries in offsets
        self.length = len(self.offsets)

        # Decide if you want (n_particles, 1000, 2) or (1000, n_particles, 2)
        self.transpose_to_time_first = transpose_to_time_first

    def __len__(self):
        print(self.dataset['position'].shape)
        return self.length


    def __getitem__(self, idx):
        idx_str = str(idx)
        offset_info = self.offsets[idx_str]
        # Extract shape info: [n_particles, 1000, 2]
        n_particles, time_steps, dim = offset_info["position"]["shape"]
        pos_offset = offset_info["position"]["offset"]

        # How many floats to slice
        chunk_size = n_particles * time_steps * dim

        chunk_size = int(chunk_size /2000)
        pos_offset = int(pos_offset/2000)
        # Slice from the 'position' array
        raw_positions = self.dataset["position"][pos_offset : pos_offset + chunk_size]
        # Reshape EXACTLY as offsets say: (n_particles, 1000, 2)
        raw_positions = raw_positions.view(n_particles, time_steps, dim)

        # If we want time first, transpose(0, 1) => (1000, n_particles, 2)
        if self.transpose_to_time_first:
            positions = raw_positions.transpose(0, 1)
        else:
            positions = raw_positions

        # Convert to float32
        positions = positions.to(torch.float32)

        # Particle types: offset in 'particle_type'
        pt_offset = offset_info["particle_type"]["offset"]
        # We'll assume shape is [n_particles]
        particle_types_data = self.dataset["particle_type"][
            pt_offset : pt_offset + n_particles
        ]
        particle_types = torch.tensor(particle_types_data, dtype=torch.long)

        # Return dictionary
        return {
            "particle_type": particle_types,  # (n_particles,)
            "position":      positions,       # (1000, n_particles, 2) if transposed
        }

In [ ]:
test_rollout_pth_path = base_path + 'valid_rollout_dataset.pth'
test_rollout_offsets_path = base_path + 'valid_rollout_offsets.json'

test_long_dataset = RolloutDataset(test_rollout_pth_path, test_rollout_offsets_path)

data_rollout = test_long_dataset[0]
data_rollout

<ipython-input-66-a7a563df10d4>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.dataset = torch.load(pth_path)
<ipython-input-66-a7a563df10d4>:73: UserWarning: To cop

{'particle_type': tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5,

In [ ]:


# Assuming `positions` is already defined and contains the correct shape
if True:
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML

    # Prepare your data
    new_positions = positions.transpose(0, 1)  # Your actual data
    data =data_rollout['position'].numpy() # Convert to numpy array if it's a tensor

    # Set up the figure and axis
    fig, ax = plt.subplots()
    ax.set_xlim(np.min(data[:, :, 0]), np.max(data[:, :, 0]))
    ax.set_ylim(np.min(data[:, :, 1]), np.max(data[:, :, 1]))
    sc = ax.scatter([], [], s=10)

    # Initialization function
    def init():
        sc.set_offsets(np.empty((0, 2)))  # Empty 2D array for initialization
        return sc,

    # Update function
    def update(frame):
        offsets = data[frame]  # Extract frame data of shape (n, 2)
        sc.set_offsets(offsets)
        return sc,

    # Create the animation
    ani = FuncAnimation(
        fig, update, frames=data.shape[0], init_func=init, blit=True, interval=100
    )

    # Render and display the animation in Colab
    display(HTML(ani.to_html5_video()))

NameError: name 'positions' is not defined

In [ ]:
%matplotlib notebook

# 2. Define the GNN model

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers, layernorm=True):
    assert n_layers>=2
    super().__init__()
    self.MLP=nn.ModuleList()
    self.MLP.append(nn.Linear(input_size, hidden_size))
    self.MLP.append(nn.ReLU())
    for i in range(1, n_layers-1):
      if i == n_layers-2:
        self.MLP.append(nn.Linear(hidden_size, output_size))
      else:
        self.MLP.append(nn.Linear(hidden_size, hidden_size))
        self.MLP.append(nn.ReLU())

    # idk what this reset_parameters is
    # they also do LayerNorm
    self.reset_parameters()

  def reset_parameters(self):
    pass

  def forward(self, x):
    for layer in self.MLP:
      x= layer(x)
    return x

In [ ]:
class Processor(pyg.nn.MessagePassing):
  def __init__(self, hidden_size, n_layers):
    super().__init__()
    self.lin_node = MLP(hidden_size*2, hidden_size, hidden_size, n_layers)
    self.lin_edge = MLP(hidden_size*3, hidden_size, hidden_size, n_layers)

  def forward(self, x, edge_index, edge_feature):
      edge_out, aggr = self.propagate(edge_index, x=(x, x), edge_feature=edge_feature)
      node_out = self.lin_node(torch.cat((x, aggr), dim=-1))
      edge_out = edge_feature + edge_out
      node_out = x + node_out
      return node_out, edge_out

  def message(self, x_i, x_j, edge_feature):
    input = torch.cat((x_i, x_j, edge_feature), dim = -1)
    output = self.lin_edge(input)
    return output

  def aggregate(self, inputs, index, dim_size = None):
    logging.info(f'aggregate inputs shape, {inputs.shape}') # n_edges, n_edge features  <-128
    logging.info(f'aggregate index {index}') # n_edges
    logging.info(f'aggregate index {index.shape}')
    out = torch_scatter.scatter(inputs, index, dim=self.node_dim, dim_size=dim_size, reduce="sum") # this I need to understand still
    return (inputs, out)

In [ ]:
import torch.nn as nn

class LearnedSimulator(nn.Module):
  def __init__(self,
               n_particle_types,
               embedding_dim,
               hidden_dimensions,
               n_layers = 3,
               n_mp_layers = 3,
               window_size = 6,
               dim = 2
               ):
    super().__init__()
    self.type_embeds = nn.Embedding(n_particle_types, embedding_dim)
    self.node_preprocess = MLP(embedding_dim + dim * (window_size -1 + 2), hidden_dimensions, hidden_dimensions, n_layers)
    self.edge_preprocess = MLP(dim+1, hidden_dimensions, hidden_dimensions, n_layers)
    self.node_postprocess = MLP(hidden_dimensions, hidden_dimensions, dim, n_layers, layernorm = False)
    self.n_mp_layers = n_mp_layers
    self.window_size = 6
    self.layers = torch.nn.ModuleList()
    for _ in range (self.n_mp_layers):
      self.layers.append(Processor(hidden_dimensions, hidden_dimensions))

  ## Reminder: graph.x -> size = (n_nodes), graph.pos -> size = ((n_nodes,14)), graph.

  def forward(self, graph):
    type_embedded = self.type_embeds(graph.x)
    node_inputs= torch.cat((type_embedded, graph.node_features), dim = -1)
    logging.info(f'Shape of node input, {node_inputs.shape}')
    node_processed = self.node_preprocess(node_inputs)
    edge_processed = self.edge_preprocess(graph.edge_features)
    logging.info(f'node_processed, {node_processed.shape}')
    logging.info(f'edge_processed,{edge_processed.shape}')
    logging.info(f'index, {graph.edge_index}')
    for processor_layer in self.layers:
      node_processed, edge_processed = processor_layer(node_processed, graph.edge_index, edge_processed)
    node_decoded = self.node_postprocess(node_processed)
    return node_decoded

In [ ]:
simulator = LearnedSimulator(9, METADATA['embedding_dimension'],METADATA['hidden_dimension'])

wandb.init(project="GNN_for_physics")

In [ ]:
## Try forwarding the data through the simulator once:
logging.info(f'{graph.x}')
out=simulator(graph)
logging.info(f'this is out shape: {out.shape}')

# 3. Perform Training

In [ ]:
def rollout(model, data, metadata, noise_std = 0):
    device = next(model.parameters()).device
    model.eval()
    window_size = model.window_size + 1
    total_time = data["position"].size(0)
    traj = data["position"][:window_size]
    traj = traj.permute(1, 0, 2)
    particle_type = data["particle_type"]

    for time in range(total_time - window_size):
        with torch.no_grad():
            graph = preprocess(particle_type, traj[:, -window_size:], None, metadata, 0.0)
            graph = graph.to(device)
            acceleration = model(graph).cpu()
            acceleration = acceleration * torch.sqrt(torch.tensor(metadata["acc_std"]) ** 2 + noise_std ** 2) + torch.tensor(metadata["acc_mean"])

            recent_position = traj[:, -1]
            recent_velocity = recent_position - traj[:, -2]
            new_velocity = recent_velocity + acceleration
            new_position = recent_position + new_velocity
            traj = torch.cat((traj, new_position.unsqueeze(1)), dim=1)

    return traj


def rolloutMSE(simulator, dataset, noise):
    total_loss = 0.0
    batch_count = 0
    simulator.eval()
    with torch.no_grad():
        for rollout_data in dataset:
            rollout_out = rollout(simulator, rollout_data, dataset.metadata, noise)
            rollout_out = rollout_out.permute(1, 0, 2)
            loss = (rollout_out - rollout_data["position"]) ** 2
            loss = loss.sum(dim=-1).mean()
            total_loss += loss.item()
            batch_count += 1
    return total_loss / batch_count

In [ ]:
wandb.finish()
wandb.init(project="GNN_for_physics")

loss,█▅▅▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
memory allocated,▂▄▅▄▅▄▄▄▄▂▄▅▃▃▃▆▅▄▆▃▅▇▄▅▅▄▄▃▄▅█▄▄▃▄▃▃▂▄▁
memory reserved,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████
n_particles,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test avg loss,▁███████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.0
lr,0.005
memory allocated,5599297536
memory reserved,9995026432
n_particles,1446


In [ ]:
valid_short_dataset[3290]

In [ ]:
# Define the DataLoader
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import ExponentialLR, StepLR

simulator = simulator.cuda()
DataLoader = pyg.data.DataLoader
first_dataloader = DataLoader(dataset = valid_short_dataset, batch_size = METADATA['batch_size'], shuffle = True)

valid_dataloader= DataLoader(dataset = valid_short_dataset, batch_size = METADATA['batch_size'])
test_dataloader= DataLoader(dataset = test_short_dataset, batch_size = METADATA['batch_size'], shuffle = False)

loss_function = nn.MSELoss()
optimizer = optim.Adam(params = simulator.parameters(),lr = METADATA['learning_rate'])
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.01 ** (1 / 500))


for epoch in range(METADATA['epochs']):
  simulator.train()
  progress_bar = tqdm(first_dataloader, desc=f"Epoch {epoch+1}", unit="batch", total=len(first_dataloader))

  avg_loss = 0

  for i, data in enumerate(progress_bar):
    optimizer.zero_grad()
    n_particles = len(data.x)
    if i==762:
      break
    if n_particles >2000: # I found that I can't load  graphs with n_particles > 2000 wiht batch_size = 4  on the GPU
      continue
    data = data.cuda()
    out = simulator(data)
    loss = loss_function(out, data.y)
    memory_allocated = torch.cuda.memory_allocated(device=None)
    memory_reserved = torch.cuda.memory_reserved(device=None)

    #Perfrom short-window evaluation on the test_set
    loss.backward()
    optimizer.step()
    #scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    to_log = {"loss": loss.item(),'n_particles': n_particles, "memory allocated": memory_allocated, "memory reserved": memory_reserved, "test avg loss": avg_loss, 'lr': current_lr }
    progress_bar.set_postfix(to_log)
    wandb.log(to_log)

    if i % 100 == 0:  # Perform evaluation every 100 steps
      simulator.eval()
      test_losses = []
      for test_step, test_data in enumerate(test_dataloader):
          if test_step == 25: # temporary measure to see it work
              break
          n_particles = len(test_data.x)
          if n_particles > 2000:
              continue
          test_data = test_data.cuda()
          test_out = simulator(test_data)
          test_loss = loss_function(test_out, test_data.y)
          test_losses.append(test_loss.item())
      avg_loss = sum(test_losses) / len(test_losses) if test_losses else 0
      print()
      print(avg_loss)
      simulator.train()

    #2. TODO: Perform rollout and calculate loss


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Epoch 1:   1%|          | 1/191 [00:09<31:29,  9.94s/batch, loss=0.00212, n_particles=1928, memory allocated=7.55e+9, memory reserved=1e+10, test avg loss=0, lr=0.005]


0.1696605432033539


Epoch 1:  53%|█████▎    | 101/191 [01:43<05:18,  3.54s/batch, loss=7.16e-7, n_particles=1928, memory allocated=6.63e+9, memory reserved=1e+10, test avg loss=0.17, lr=0.005]


3.980049353913273e-07


Epoch 1: 100%|██████████| 191/191 [02:54<00:00,  1.09batch/s, loss=6.5e-8, n_particles=1446, memory allocated=6.66e+9, memory reserved=1e+10, test avg loss=3.98e-7, lr=0.005]


In [ ]:
def rollout(model, data, metadata, noise_std = 0):
    device = next(model.parameters()).device
    model.eval()
    window_size = model.window_size
    total_time = data["position"].size(0)
    traj = data["position"][:window_size]
    traj = traj.permute(1, 0, 2)
    particle_type = data["particle_type"]

    for time in range(1500):
        with torch.no_grad():
            graph = preprocess(particle_type, traj[:, -window_size:], metadata)
            graph = graph.to(device)
            acceleration = model(graph).cpu()
            acceleration = acceleration * torch.sqrt(torch.tensor(metadata["acc_std"]) ** 2 + noise_std ** 2) + torch.tensor(metadata["acc_mean"])

            recent_position = traj[:, -1]
            recent_velocity = recent_position - traj[:, -2]
            new_velocity = recent_velocity + acceleration
            new_position = recent_position + new_velocity
            traj = torch.cat((traj, new_position.unsqueeze(1)), dim=1)

    return traj


def oneStepMSE(simulator, dataloader, metadata, noise):
    """Returns two values, loss and MSE"""
    total_loss = 0.0
    total_mse = 0.0
    batch_count = 0
    simulator.eval()
    with torch.no_grad():
        scale = torch.sqrt(torch.tensor(metadata["acc_std"]) ** 2 + noise ** 2).cuda()
        for data in valid_loader:
            data = data.cuda()
            pred = simulator(data)
            mse = ((pred - data.y) * scale) ** 2
            mse = mse.sum(dim=-1).mean()
            loss = ((pred - data.y) ** 2).mean()
            total_mse += mse.item()
            total_loss += loss.item()
            batch_count += 1
    return total_loss / batch_count, total_mse / batch_count


def rolloutMSE(simulator, dataset, noise):
    total_loss = 0.0
    batch_count = 0
    simulator.eval()
    with torch.no_grad():
        for rollout_data in dataset:
            rollout_out = rollout(simulator, rollout_data, dataset.metadata, noise)
            rollout_out = rollout_out.permute(1, 0, 2)
            loss = (rollout_out - rollout_data["position"]) ** 2
            loss = loss.sum(dim=-1).mean()
            total_loss += loss.item()
            batch_count += 1
    return total_loss / batch_count

## Visualize one rollout with the model

In [ ]:
#testing the test dataloader
first_window = data_rollout
rollout_traj = rollout(simulator, first_window, METADATA)


In [ ]:
transposed_rollout_traj = rollout_traj.transpose(0,1)

In [ ]:
# Create example data: Random walk with (timesteps, n, 2)
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

n = 295  # Number of points
timesteps =1000 # Number of frames

data = transposed_rollout_traj.numpy()  # Random walk data

# Set up the figure and axis
fig, ax = plt.subplots()
ax.set_xlim(0.0, 0.9)
ax.set_ylim(np.min(data[:, :, 1]), 0.9)
sc = ax.scatter([], [], s=10)

# Initialization function
def init():
    sc.set_offsets(np.empty((0, 2)))  # Empty 2D array for initialization
    return sc,

# Update function
def update(frame):
    offsets = data[frame]  # Extract frame data of shape (n, 2)
    sc.set_offsets(offsets)
    return sc,

# Create the animation
ani = FuncAnimation(
    fig, update, frames=timesteps, init_func=init, blit=True, interval=10
)

HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>

## Clarification:
This notebook was made as a proof of concept. The objective was to create the datasets, build up the architecture, and see the loss fall during training. A simple heuristic for the water was learned: water falling down. This was performed by training on the validation and testing on the test dataset.

Too see the model reproduce the dynamics of fluid splashing, one would need to train it on the whole train dataset. This would strain my local machine during data preprocessing, and require many hours of compute during training. If I find more time in the future, I will complete it.